In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz

     |████████████████████████████████| 827.9 MB 4.4 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.0-py3-none-any.whl size=829187827 sha256=6553b8da65bc9c2421089a6e2aef25d3a9157fbbe7a0338ae34fa022ff3c8a76
  Stored in directory: /root/.cache/pip/wheels/4b/6e/87/8b129988ba4d66c064415e09db36d5945b638aabcaf8f41765
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.0 MB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529.csv	 Github
 a0530.csv	 heatmap530.ipynb
 a0531.csv	 heatmap531.ipynb
 a0601.csv	 Heat_Map_Online_language.ipynb
 a0602.csv	 __MACOSX
 a0603.csv	 non_violant1.csv
 a0604.csv	 non_violant2.csv
 csv		 violant.csv
 csv.zip	'新建 Microsoft PowerPoint 演示文稿.pptx'
 dataframe_csv


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [11]:
ct_path_605 = 'csv/US06_05.csv'

In [12]:
ct = pd.read_csv(ct_path_605)

In [13]:
ct.shape[1]

47

In [14]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [15]:
#len(columns1)
len(columns2)

47

In [16]:
ct.columns=columns2

In [17]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,28071939336_10158264308754337,Facebook,2020-06-05 21:47:20,2021-08-04 17:55:24,photo,Joe Biden Denounces Donald Trump’s Invocation ...,devorado.com,"On Friday, the Democratic Party’s presumptive ...",Me and a couple thousand kids paddled out 2 da...,1.0,https://www.facebook.com/diplo/photos/a.396614...,https://www.facebook.com/28071939336/posts/101...,3530886,-1.454259,1.0,1246,32,19,282,3,3,0,0,0,42,1703,49,68,405,18,56,3,3,0,19,79720,Diplo,diplo,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,hey i'm diplo\n\nhttps://linktr.ee/diplo,True,2008-06-04 20:55:36,en,0,79720|10158264308754337
1,19125738813_10158472465998814,Facebook,2020-06-05 21:47:17,2021-08-31 03:56:04,link,"Evansville Police, Vanderburgh Sheriff honor G...",courierpress.com,Evansville Police Department and Vanderburgh C...,Bolin ended the 8 minutes and 46 seconds of si...,1.0,https://www.courierpress.com/story/news/2020/0...,https://www.facebook.com/19125738813/posts/101...,53407,10.170213,1.0,157,215,30,71,2,0,3,0,0,17,12,6,6,5,3,3,7,5,0,2,25884,Courier & Press,courierpress,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,"The Courier & Press in Evansville, Indiana, is...",True,2008-07-14 16:30:17,en,0,25884|10158472465998814
2,5550296508_10160941256141509,Facebook,2020-06-05 21:47:11,2021-08-31 17:16:18,link,Fact check: Trump glosses over racial disparit...,cnn.com,Evansville Police Department and Vanderburgh C...,"Facts First: Trump's comment about ""equality"" ...",1.0,https://cnn.it/2Y5bZ3h,https://www.facebook.com/5550296508/posts/1016...,37215383,2.926015,1.0,2450,899,2772,22,150,212,519,3931,0,1,1739,211,975,97,101,173,295,153,0,31,8323,CNN,cnn,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Instant breaking news alerts and the most talk...,True,2007-11-07 22:14:27,en,0,8323|10160941256141509
3,491752947643347_1602681509883813,Facebook,2020-06-05 21:47:08,2021-07-17 06:23:23,link,Artists Paint George Floyd Mural at Cup Foods,mspmag.com,"Xena Goldman, Greta McLain, and Cadex Herrera ...",Thank you Artists of the world❤️❤️❤️🎨 StrongTo...,1.0,http://mspmag.com/arts-and-culture/artists-pai...,https://www.facebook.com/491752947643347/posts...,0,-10.000000,1.0,2,0,0,0,0,0,0,0,0,0,6,4,4,5,1,0,0,0,0,1,12252864,Face Off Theatre Company,FaceOffTheatre,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Face Off Theatre Company is based in Kalamazoo...,False,2015-06-08 17:19:57,en,0,12252864|1602681509883813
4,132880973412246_3356194254414219,Facebook,2020-06-05 21:47:07,2021-09-27 23:18:27,photo,Artists Paint George Floyd Mural at Cup Foods,mspmag.com,"Xena Goldman, Greta McLain, and Cadex Herrera ...",Our Three Squared team stands by BIPOC as alli...,1.0,https://www.facebook.com/ThreeSquaredInc/photo...,https://www.facebook.com/132880973412246/posts...,0,-1.705882,1.0,9,0,1,6,0,1,0,0,0,0,14,5,5,3,2,0,0,0,0,0,14142972,"Three Squared, Inc",ThreeSquaredInc,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,"Building faster, stronger and more cost effect...",False,2010-07-06 16:09:18,en,0,14142972|3356194254414219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [18]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [19]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [20]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [21]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [22]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [23]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [24]:
ct['NER-msg'] = ct['message'].apply(ner)

In [25]:
ct['NER-desc'] = ct['description'].apply(ner)

In [26]:
ct['NER-title'] = ct['title'].apply(ner)

In [27]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,{'Encinitas': 1},{}
1,{},{}
2,{},{}
3,{},{}
4,{},{}


In [32]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,28071939336_10158264308754337,Facebook,2020-06-05 21:47:20,2021-08-04 17:55:24,photo,Joe Biden Denounces Donald Trump’s Invocation ...,devorado.com,"On Friday, the Democratic Party’s presumptive ...",Me and a couple thousand kids paddled out 2 da...,1.0,https://www.facebook.com/diplo/photos/a.396614...,https://www.facebook.com/28071939336/posts/101...,3530886,-1.454259,1.0,1246,32,19,282,3,3,0,0,0,42,1703,49,68,405,18,56,3,3,0,19,79720,Diplo,diplo,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,hey i'm diplo\n\nhttps://linktr.ee/diplo,True,2008-06-04 20:55:36,en,0,79720|10158264308754337,{'Encinitas': 1},{},{}
1,19125738813_10158472465998814,Facebook,2020-06-05 21:47:17,2021-08-31 03:56:04,link,"Evansville Police, Vanderburgh Sheriff honor G...",courierpress.com,Evansville Police Department and Vanderburgh C...,Bolin ended the 8 minutes and 46 seconds of si...,1.0,https://www.courierpress.com/story/news/2020/0...,https://www.facebook.com/19125738813/posts/101...,53407,10.170213,1.0,157,215,30,71,2,0,3,0,0,17,12,6,6,5,3,3,7,5,0,2,25884,Courier & Press,courierpress,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,"The Courier & Press in Evansville, Indiana, is...",True,2008-07-14 16:30:17,en,0,25884|10158472465998814,{},{},{}
2,5550296508_10160941256141509,Facebook,2020-06-05 21:47:11,2021-08-31 17:16:18,link,Fact check: Trump glosses over racial disparit...,cnn.com,Evansville Police Department and Vanderburgh C...,"Facts First: Trump's comment about ""equality"" ...",1.0,https://cnn.it/2Y5bZ3h,https://www.facebook.com/5550296508/posts/1016...,37215383,2.926015,1.0,2450,899,2772,22,150,212,519,3931,0,1,1739,211,975,97,101,173,295,153,0,31,8323,CNN,cnn,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,Instant breaking news alerts and the most talk...,True,2007-11-07 22:14:27,en,0,8323|10160941256141509,{},{},{}
3,491752947643347_1602681509883813,Facebook,2020-06-05 21:47:08,2021-07-17 06:23:23,link,Artists Paint George Floyd Mural at Cup Foods,mspmag.com,"Xena Goldman, Greta McLain, and Cadex Herrera ...",Thank you Artists of the world❤️❤️❤️🎨 StrongTo...,1.0,http://mspmag.com/arts-and-culture/artists-pai...,https://www.facebook.com/491752947643347/posts...,0,-10.000000,1.0,2,0,0,0,0,0,0,0,0,0,6,4,4,5,1,0,0,0,0,1,12252864,Face Off Theatre Company,FaceOffTheatre,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Face Off Theatre Company is based in Kalamazoo...,False,2015-06-08 17:19:57,en,0,12252864|1602681509883813,{},{},{}
4,132880973412246_3356194254414219,Facebook,2020-06-05 21:47:07,2021-09-27 23:18:27,photo,Artists Paint George Floyd Mural at Cup Foods,mspmag.com,"Xena Goldman, Greta McLain, and Cadex Herrera ...",Our Three Squared team stands by BIPOC as alli...,1.0,https://www.facebook.com/ThreeSquaredInc/photo...,https://www.facebook.com/132880973412246/posts...,0,-1.705882,1.0,9,0,1,6,0,1,0,0,0,0,14,5,5,3,2,0,0,0,0,0,14142972,"Three Squared, Inc",ThreeSquaredInc,https://scontent-sea1-1.xx.fbcdn.net/v/t31.181...,facebook_page,US,"Building faster, stronger and more cost effect...",False,2010-07-06 16:09:18,en,0,14142972|3356194254414219,{},{},{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [30]:
ct.to_csv('a0605.csv')

In [36]:
import csv
with open(r"a0605.csv", 'r') as f:
    reader = csv.reader(f)
    linenumber = 1
    try:
        for row in reader:
            linenumber += 1
    except Exception as e:
        print (("Error line %d: %s %s" % (linenumber, str(type(e)), e)))

In [39]:
ct=pd.read_csv('a0605.csv',lineterminator='\n')

In [40]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [41]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [42]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{'Encinitas': 1},{},{}
7,"{'Troy': 1, 'Metro Detroit': 1}",{},{}
11,{'Buffalo': 2},{},{}
12,{'DC': 1},{},{}
15,{},"{'Minnesota': 1, 'Sa-Roc': 1, 'Atlanta': 1, 'S...",{}


In [43]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [44]:
ct_ner['NER-msg']

0                  [Encinitas]
7        [Troy, Metro Detroit]
11                   [Buffalo]
12                        [DC]
15                          []
                 ...          
18670                [El Paso]
18671             [New Canaan]
18672                       []
18674             [Washington]
18675                       []
Name: NER-msg, Length: 12556, dtype: object

In [45]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
ct_ner['NER']

0                                            [Encinitas]
7                                  [Troy, Metro Detroit]
11                                             [Buffalo]
12                                                  [DC]
15       [Sa-Roc, Minnesota, St. Louis, Newark, Atlanta]
                              ...                       
18670                 [El Paso, El PASO, El Paso, Texas]
18671              [New Canaan, El PASO, El Paso, Texas]
18672                                [Chicago, Illinois]
18674               [Washington, Washington, Washington]
18675                           [Washington, Washington]
Name: NER, Length: 12556, dtype: object

In [47]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
ner_lst = ct_ner['NER'].values

In [50]:
ner_lst

array([list(['encinitas']), list(['troy', 'metro detroit']),
       list(['buffalo']), ..., list(['chicago', 'illinois']),
       list(['washington']), list(['washington'])], dtype=object)

In [51]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [52]:
list

['encinitas',
 'troy',
 'metro detroit',
 'buffalo',
 'dc',
 'sa-roc',
 'minnesota',
 'st. louis',
 'newark',
 'atlanta',
 'sa-roc',
 'minnesota',
 'st. louis',
 'newark',
 'atlanta',
 'philadelphia',
 'new legislature to stop',
 'u.s.',
 'lafayette park',
 'washington',
 'afghanistan',
 'pakistan',
 'n.c.',
 'n.y.',
 'america',
 'north korea',
 'fort bragg',
 'fort drum',
 'u.s.',
 'lafayette park',
 'washington',
 'us',
 'afghanistan',
 'pakistan',
 'n.c.',
 'n.y.',
 'america',
 'north korea',
 'fort bragg',
 'fort drum',
 'u.s.',
 'lafayette park',
 'washington',
 'afghanistan',
 'pakistan',
 'n.c.',
 'n.y.',
 'america',
 'north korea',
 'fort bragg',
 'fort drum',
 'u.s.',
 'lafayette park',
 'washington',
 'afghanistan',
 'pakistan',
 'n.c.',
 'n.y.',
 'america',
 'north korea',
 'fort bragg',
 'fort drum',
 'tx',
 'austin',
 'texas',
 'tx',
 'austin',
 'texas',
 'tx',
 'austin',
 'texas',
 'nevada',
 'nevada',
 'seattle',
 'us',
 'the united states',
 'madison',
 'wisconsin',
 'w

In [53]:
c = collections.Counter()

In [54]:
dict_a=collections.Counter(list).most_common()

In [55]:
dict_a

[('minneapolis', 4133),
 ('america', 1268),
 ('u.s.', 815),
 ('minnesota', 750),
 ('texas', 632),
 ('washington', 589),
 ('the united states', 589),
 ('us', 523),
 ('houston', 497),
 ('new york', 404),
 ('chicago', 382),
 ('new york city', 337),
 ('d.c.', 322),
 ('los angeles', 275),
 ('maryland', 250),
 ('buffalo', 220),
 ('california', 219),
 ('florida', 171),
 ('atlanta', 170),
 ('nyc', 148),
 ('dc', 146),
 ('dallas', 138),
 ('ahmaud arbery', 132),
 ('north carolina', 132),
 ('new jersey', 129),
 ('georgia', 129),
 ('michigan', 128),
 ('detroit', 127),
 ('portland', 127),
 ('denver', 119),
 ('seattle', 118),
 ('alabama', 108),
 ('louisville', 102),
 ('philadelphia', 95),
 ('brooklyn', 92),
 ('boston', 84),
 ('nevada', 81),
 ('wisconsin', 80),
 ('mn', 80),
 ('oakland', 80),
 ('hollywood', 78),
 ('nj', 77),
 ('orange county', 76),
 ('milwaukee', 75),
 ('st. louis', 74),
 ('iowa', 73),
 ('virginia', 69),
 ('kentucky', 68),
 ('arizona', 68),
 ('london', 66),
 ('illinois', 62),
 ('las ve

In [56]:
dataframe605=pd.DataFrame(dict_a)

In [57]:
len(dict_a)

2629

In [58]:
dataframe605.to_csv('dataframe0605.csv')